### Importación de librerias

In [105]:
import pandas as pd
from torch.utils.data import Dataset
import torch
from torch.utils.data import random_split
import torch.nn.functional as F
import torch.nn as nn
import torch
from torchmetrics import MeanSquaredError, MeanAbsoluteError, R2Score

### Cargamos los datos

In [106]:
airfoilDataset = pd.read_csv('/media/DIURNO/PIA/PyTorch/Tarea70/airfoil_self_noise.dat', sep='\t',names=['Frequency (Hz)', 'Angle of attack (ºC)', 'Chord length (m)', 'Free-stream velocity (m/s)', 'Suction side displacement thickness (m)', 'pressure level (dB)'])
X = airfoilDataset.loc[:, ~airfoilDataset.columns.isin(['pressure level (dB)'])]
Y = airfoilDataset[["pressure level (dB)"]]
airfoilDataset

,Frequency (Hz),Angle of attack (ºC),Chord length (m),Free-stream velocity (m/s),Suction side displacement thickness (m),pressure level (dB)
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


### Transformaciones

In [107]:
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

In [108]:
class AirfoilDataset(Dataset):
  def __init__(self, src_file, root_dir, transform=None):
    airfoilDataset = pd.read_csv(src_file, sep='\t',names=['Frequency (Hz)', 'Angle of attack (ºC)', 'Chord length (m)', 'Free-stream velocity (m/s)', 'Suction side displacement thickness (m)', 'pressure level (dB)'])
    X = airfoilDataset.loc[:, ~airfoilDataset.columns.isin(['pressure level (dB)'])]
    Y = airfoilDataset[["pressure level (dB)"]]

    s1=X.iloc[:,0:5].values
    x_tensor = torch.tensor(s1)
    
    y_tensor = torch.tensor(Y.values).type(torch.float32)
    
    scaler = StandardScaler()
    XScalada = scaler.fit_transform(x_tensor).type(torch.float32)

    self.data = torch.cat((XScalada,y_tensor),1)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    preds = self.data[idx, 0:5]
    spcs = self.data[idx, 5:]
    sample = (preds, spcs)
    if self.transform:
      sample = self.transform(sample)
    return sample

### Cargamos los datos

In [109]:
dataset = AirfoilDataset("airfoil_self_noise.dat",".")
display(dataset[0])

(tensor([-0.6618, -1.1460,  1.7987,  1.3125, -0.6446]), tensor([126.2010]))

### División train y test

In [110]:
lonxitudeDataset = len(dataset)
tamTrain =int(lonxitudeDataset*0.8)
tamVal = lonxitudeDataset - tamTrain
print(f"Tam dataset: {lonxitudeDataset} train: {tamTrain} tamVal: {tamVal}")
train_set, val_set = random_split(dataset,[tamTrain,tamVal])
train_ldr = torch.utils.data.DataLoader(train_set, batch_size=2,
    shuffle=True, drop_last=False)
validation_loader =torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False, drop_last=True)

Tam dataset: 1503 train: 1202 tamVal: 301


### Creación del modelo

In [111]:
class Model(nn.Module):
    def __init__(self, entradas):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(entradas, 100)
        self.layer2 = nn.Linear(100, 50)
        self.layer3 = nn.Linear(in_features=50, out_features=1)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        return x

### Instanciamos el modelo

In [112]:
model     = Model(5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn   = nn.MSELoss(reduction='sum')
display(model)

Model(
  (layer1): Linear(in_features=5, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=1, bias=True)
)

In [113]:
entradaProba,dest = next(iter(train_ldr))
print("Entrada:")
display(entradaProba)
print("Desexada:")
display(dest)
saida = model(entradaProba) 
print("Saída:")
display(saida)
loss_fn(saida, dest)

Entrada:


tensor([[-0.7157, -0.4701,  1.7987, -0.7231, -0.4063],
        [-0.5191, -0.6391,  1.7987,  0.2979, -0.5030]])

Desexada:


tensor([[131.0730],
        [127.3290]])

Saída:


tensor([[0.1674],
        [0.1877]], grad_fn=<ReluBackward0>)

tensor(33301.1875, grad_fn=<MseLossBackward0>)

### Función de entrenamiento

In [114]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    for i, data in enumerate(train_ldr):

        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_ldr)

In [115]:
from torch.utils.tensorboard import SummaryWriter

In [116]:
EPOCHS = 200
writer = None
tb = SummaryWriter()
for epoch in range(EPOCHS):
    model.train(True)
    avg_loss = train_one_epoch(epoch, tb)

    mean_squared_error = MeanSquaredError()
    mean_absolute_error = MeanAbsoluteError()
    r2Score = R2Score()
    model.train(False)

    with torch.no_grad():
        for entradas, saidas in validation_loader:
            voutputs = model(entradas)
            mean_squared_error(voutputs,saidas)
            mean_absolute_error(voutputs,saidas)
            r2Score(voutputs,saidas)

    errorMedio = mean_squared_error.compute()
    errorAbsolute =mean_absolute_error.compute()
    r2 = r2Score.compute()

    tb.add_scalar('Promedio', avg_loss, epoch)
    tb.add_scalar('Error Medio', errorMedio, epoch)
    tb.add_scalar('Error Absoluto', errorAbsolute, epoch)
    tb.add_scalar('r2', r2, epoch)